In [1]:
print("hello world")

hello world


In [2]:
import pandas as pd
import numpy as np


In [18]:


# Read CSV file
df = pd.read_csv("train.csv")

# See first few rows
print(df.head())


   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [25]:
data = df.select_dtypes(include=['number']).to_numpy()
print(np.shape(data))

(1460, 38)


In [72]:
from sklearn.impute import SimpleImputer



# Impute missing values with column means
imputer = SimpleImputer(strategy="mean")
z = imputer.fit_transform(data)

# Separate features and target
y = z[:, -1]   # assuming last column is SalePrice
x = z[:, 1:-1]

print("Any NaN left in x?", np.isnan(x).any())
print("Any NaN left in y?", np.isnan(y).any())


Any NaN left in x? False
Any NaN left in y? False


In [73]:

n = x.shape[1]
init_w = np.full((n,),0.8)
init_b = 0.8
print(x.shape)
print(y.shape)


(1460, 36)
(1460,)


In [74]:
print(init_w.shape)
print(x)



(36,)
[[6.000e+01 6.500e+01 8.450e+03 ... 0.000e+00 2.000e+00 2.008e+03]
 [2.000e+01 8.000e+01 9.600e+03 ... 0.000e+00 5.000e+00 2.007e+03]
 [6.000e+01 6.800e+01 1.125e+04 ... 0.000e+00 9.000e+00 2.008e+03]
 ...
 [7.000e+01 6.600e+01 9.042e+03 ... 2.500e+03 5.000e+00 2.010e+03]
 [2.000e+01 6.800e+01 9.717e+03 ... 0.000e+00 4.000e+00 2.010e+03]
 [2.000e+01 7.500e+01 9.937e+03 ... 0.000e+00 6.000e+00 2.008e+03]]


In [84]:

m = len(x)
print(m)
y_pred = np.zeros(m)
def predict(X, w, b):
    return np.dot(X, w) + b
            


1460


In [85]:
def calc_cost(actual,prediction):
    c=0
    cost = 0
    for i in range(m):
        c+= (actual[i] - prediction[i])**2
    cost = (1/(2*m))*c
    return cost

In [86]:
y_pred = predict(x, init_w, init_b)

print(y)
print(y_pred)

J = calc_cost(y, y_pred)

[208500. 181500. 223500. ... 266500. 142125. 147500.]
[18049.6 18781.6 20522.4 ... 21530.4 18214.4 19256.8]


Formula for Gradient:

djdw = 1/m sum(((wx_i + b) - yi)xi)
djdb = 1/m sum(((wx_i + b) - yi ))

In [87]:
def compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_dw, dj_db

In [88]:
a, b = compute_gradient(x,y,init_w,init_b)

In [93]:
print(a)
print(b)



-161264.79730378836
[-8.94366891e+06 -1.18131957e+07 -1.82050295e+09 -1.06776116e+06
 -8.92518681e+05 -3.19127704e+08 -3.20905439e+08 -2.32356313e+07
 -8.44485874e+07 -7.20799328e+06 -9.87680925e+07 -1.90424673e+08
 -2.04684564e+08 -6.66042582e+07 -8.38534829e+05 -2.72127357e+08
 -7.71326531e+04 -8.87146002e+03 -2.75847300e+05 -7.28215764e+04
 -4.71946557e+05 -1.66422912e+05 -1.11578460e+06 -1.20869443e+05
 -3.19949874e+08 -3.21336354e+05 -8.63321361e+07 -1.81871283e+07
 -9.10291949e+06 -2.93030874e+06 -6.46767900e+05 -2.89121981e+06
 -7.02433770e+05 -5.84333105e+06 -1.02927890e+06 -3.23787165e+08]
